# Problem: Lemonade Change

Statement
There is a lemonade stand where customers can buy one lemonade at a time for 
$
5
$5
 and pay with a 
$
5
$5
, 
$
10
$10
, or 
$
20
$20
 bill. It is necessary to return the correct change to each customer so that the net transaction is completed successfully with a total payment of 
$
5
$5
. Note that no change is available initially.

Given an integer array, bills, where bills[i] represents the bill paid by the 
i
t
h
i 
th
 
 customer, return TRUE if it is possible to provide every customer with the correct change, or FALSE otherwise.

Constraints:

1
≤
1≤
 bills.length 
≤
500
≤500

bills[i] is either 
5
5
, 
10
10
, or 
20
20
.

Hint:
- You might want to go over the Greedy Techniques pattern again.

In [ ]:
from collections import defaultdict

def lemonade_change(bills):
    available_bills = defaultdict(int)
    
    for bill in bills:
        if bill == 5:
            available_bills[5] += 1
        
        elif bill == 10:
            if available_bills[5] == 0:
                return False
            available_bills[5] -= 1
            available_bills[10] += 1
        
        elif bill == 20:
            if available_bills[10] > 0 and available_bills[5] > 0:
                available_bills[10] -= 1
                available_bills[5] -= 1
            elif available_bills[5] >= 3:
                available_bills[5] -= 3
            else:
                return False
    
    return True


In [5]:
bills = [5,5,5,5,20,10,10]
lemonade_change(bills)

False

In [ ]:
# Better optimized solution
def lemonade_change(bills):
    five, ten = 0, 0   # counters for $5 and $10 bills we have
    
    for bill in bills:
        if bill == 5:
            five += 1
        elif bill == 10:
            if five == 0:
                return False
            five -= 1
            ten += 1
        else:  # bill == 20
            if ten > 0 and five > 0:
                ten -= 1
                five -= 1
            elif five >= 3:
                five -= 3
            else:
                return False
    
    return True


# Problem: Finding MK Average

You are given two integers, m and k, and a stream of integers. Your task is to design and implement a data structure that efficiently calculates the MK Average for the stream.

To compute the MK Average, follow these steps:

Stream length check: If the stream contains fewer than m elements, return -1 as the MK Average.

Window selection: Otherwise, copy the last m elements of the stream to a separate container and remove the smallest k elements and the largest k elements from the container.

Average calculation: Calculate the average of the remaining elements (rounded down to the nearest integer).

Implement the MKAverage class

MKAverage(int m, int k): Initializes the object with integers m and k and an empty stream.

void addElement(int num): Adds the integer num to the stream.

int calculateMKAverage(): Returns the current MK Average for the stream as described above, or -1 if the stream contains fewer than m elements.

Constraints:

3
<
=
3<=
 m 
<
=
1
0
5
<=10 
5
 

1
<
1<
 k*2 
<
m
<m

1
<
=
1<=
 num 
<
=
1
0
5
<=10 
5
 

1
0
3
10 
3
 
 calls will be made to addElement and calculateMKAverage, at most.

In [ ]:
from collections import deque
import bisect
class MKAverage(object):
    def __init__(self, m, k):
        self.m = m
        self.k = k
        self.container = deque()
        self.sortedList = []
        self.midSum = 0


    def addElement(self, num):
        self.container.append(num)

        bisect.insort(self.sortedList, num)

        if len(self.container)>self.m: #remove oldest from stream container and sortedList
            old = self.container.popleft()
            idx = bisect.bisect_left(self.sortedList,old)
            self.sortedList.pop(idx)
        
        if len(self.container) == self.m:
            self._compute_mid_sum()
        
    
    def _compute_mid_sum(self):
        self.midSum = sum(self.sortedList[self.k:self.m-self.k])

    
    def calculateMKAverage(self):
        if len(self.container)<self.m:
            return -1
        return self.midSum // (self.m - 2* self.k)

Explanation

- We maintain a deque (stream) to track last m elements in order.
- We maintain a sorted_list (acts as a BST):
  - Insert new number with bisect.insort (O(m)).
  - Remove oldest number using bisect.bisect_left (O(m)).
To compute MKAverage:
- Take slice sorted_list[k : m-k].
- Compute sum → average.

# Leetcode's problem 187. Repeated DNA Sequences

The DNA sequence is composed of a series of nucleotides abbreviated as 'A', 'C', 'G', and 'T'.

For example, "ACGAATTCCG" is a DNA sequence.
When studying DNA, it is useful to identify repeated sequences within the DNA.

Given a string s that represents a DNA sequence, return all the 10-letter-long sequences (substrings) that occur more than once in a DNA molecule. You may return the answer in any order.

 

Example 1:

Input: s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
Output: ["AAAAACCCCC","CCCCCAAAAA"]
Example 2:

Input: s = "AAAAAAAAAAAAA"
Output: ["AAAAAAAAAA"]
 

Constraints:

1 <= s.length <= 105
s[i] is either 'A', 'C', 'G', or 'T'.


In [ ]:
# Ali's solution: using hash tables
from collections import defaultdict
def findRepeatedDnaSequences(s):
    if len(s)<10:
        return []
    l,h = 0, 9
    n = len(s)
    seen = set()
    res = set()
    while h<n:
        
        curr = s[l:h+1]
        if curr in seen:
            res.add(curr)
        else:
            seen.add(curr)

        l+=1
        h+=1
    
    return list(res)

In [26]:
s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
findRepeatedDnaSequences(s)

['AAAAACCCCC', 'CCCCCAAAAA']

In [27]:
s = "AAAAAAAAAAAAA"
findRepeatedDnaSequences(s)

['AAAAAAAAAA']

Great — let’s add the **intuition**, the **step-by-step approach**, and inline **comments** so it’s crystal clear why this works and why it’s fast.

---

## ✨ Intuition

* The DNA alphabet has only 4 letters (`A, C, G, T`).
* Each letter can be represented in **2 bits**:

  * `A → 00`, `C → 01`, `G → 10`, `T → 11`.
* So any 10-letter DNA substring can be represented in **20 bits** (fits easily in a normal integer).
* Instead of slicing strings of length 10 at every step (which is costly), we encode them into integers and slide a rolling hash window.
* We use a bitmask to **drop old characters** when sliding and keep only the last 20 bits.
* A set tracks hashes we’ve seen; if a hash appears again, the substring is repeated.

---

## 🪜 Step-by-Step Approach

1. **Edge case:** If string is shorter than 10, no repeats possible.
2. **Initialize mapping:** `A, C, G, T` → 0,1,2,3.
3. **Build first window (10 chars):** Left-shift and append bits for each character.
4. **Store first hash in `seen`.**
5. **Slide through the rest of the string:**

   * Shift left by 2 bits, append new char bits.
   * Apply a mask `(1 << 20) - 1` to keep only 20 bits (last 10 chars).
   * If hash is already in `seen`, substring is a repeat → add it to `res`.
   * Otherwise add hash to `seen`.
6. **Return all repeats as a list.**

---

## 📘 Code with Comments

```python
class Solution:
    def findRepeatedDnaSequences(self, s: str) -> List[str]:
        # No possible repeats if shorter than 10
        if len(s) < 10:
            return []

        # Map each nucleotide to 2 bits
        bitMap = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

        # Mask to keep only 20 bits (10 chars * 2 bits)
        mask = (1 << 20) - 1  
        hash_val = 0  # rolling hash value

        seen = set()  # hashes seen once
        res = set()   # substrings seen more than once

        # Encode the first 10 characters into the initial hash
        for i in range(10):
            hash_val = (hash_val << 2) | bitMap[s[i]]
        seen.add(hash_val)

        # Slide the window over the rest of the string
        for i in range(10, len(s)):
            # Shift left by 2, add new char bits, and mask off old chars
            hash_val = ((hash_val << 2) | bitMap[s[i]]) & mask

            if hash_val in seen:
                # Found a repeat: extract substring (only when needed)
                res.add(s[i-9:i+1])
            else:
                # First time seeing this 10-char sequence
                seen.add(hash_val)

        # Convert set to list for output
        return list(res)
```

---

## 📊 Complexity

* **Time:** O(n) → one pass over the string.
* **Space:** O(n) → storing up to O(n) hashes and repeated substrings.
* **Optimization gain:** avoids creating n substrings of length 10; works mostly with integers.

---

👉 Do you want me to also show the **bit-decoding alternative** (instead of slicing `s[i-9:i+1]`) with comments, so you have both approaches fully documented?


In [ ]:
# Ali's solution using bit encoding trick (A=0, C=1, G=2, T=3) and bitmask/rolling hash:

def findRepeatedDnaSequences(s):
    if len(s)<10:
        return []
    bitMap = {'A':0, 'C':1, 'G':2, 'T':3}
    mask = (1 << 20) - 1   # 20 ones in binary
    hash_val = 0

    seen = set()
    res = set()

    for i in range(10):
        hash_val = (hash_val << 2) | bitMap[s[i]]
    seen.add(hash_val)
    for i in range(10,len(s)):
        hash_val = ((hash_val << 2) | bitMap[s[i]]) & mask
        if hash_val in seen:
            res.add(s[i-9:i+1])
        else:
            seen.add(hash_val)
    return list(res)


In [33]:
s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
findRepeatedDnaSequences(s)

['AAAAACCCCC', 'CCCCCAAAAA']

# Leetcode's Problem 416. Partition Equal Subset Sum

Given an integer array nums, return true if you can partition the array into two subsets such that the sum of the elements in both subsets is equal or false otherwise.

 

Example 1:

Input: nums = [1,5,11,5]
Output: true
Explanation: The array can be partitioned as [1, 5, 5] and [11].
Example 2:

Input: nums = [1,2,3,5]
Output: false
Explanation: The array cannot be partitioned into equal sum subsets.
 

Constraints:

1 <= nums.length <= 200
1 <= nums[i] <= 100

In [ ]:
# Brute force approach: recursion without memoization
from typing import List
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        
        def dp(nums, n, subset_sum): #dp is here doing the dfs so it is also a deep first search algorithm
            if subset_sum == 0:
                return True
            if n == 0 or subset_sum<0:
                return False
            
            return dp(nums, n-1, subset_sum) or dp(nums, n-1, subset_sum-nums[n-1])
        
        n = len(nums)
        total_sum = sum(nums)
        if total_sum % 2 != 0:
            return False
        return dp(nums, n-1, total_sum//2)
            

In [35]:
nums = [1,5,11,5]
sol = Solution()
sol.canPartition(nums)


True

In [36]:
nums = [1,2,3,5]
sol = Solution()
sol.canPartition(nums)

False

In [43]:
from typing import List
from functools import lru_cache
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        @lru_cache(None)
        def dp(n, subset_sum): #dp is here doing the dfs so it is also a deep first search algorithm
            if subset_sum == 0:
                return True
            if n == 0 or subset_sum<0:
                return False
            
            return dp(n-1, subset_sum) or dp( n-1, subset_sum-nums[n-1])
        
        n = len(nums)
        total_sum = sum(nums)
        if total_sum % 2 != 0:
            return False
        return dp( n-1, total_sum//2)

In [44]:
nums = [1,5,11,5]
sol = Solution()
sol.canPartition(nums)

True

In [ ]:
# bottom-up approach
from typing import List
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        n = len(nums)
        totalSum = sum(nums)
        if totalSum %2 != 0:
            return False
        subset_sum = totalSum // 2
        dp = [[False]*(subset_sum+1) for _ in range(n+1)]
        dp[0][0] = True
        
        for i in range(1,n+1):
            curr = nums[i-1]
            for j in range(subset_sum+1):
                if j< curr:
                    dp[i][j] = dp[i-1][j]
                else:
                    dp[i][j] = dp[i-1][j] or dp[i-1][j-curr]
        print(dp)
        return dp[n][subset_sum]



In [72]:
nums = [1,5,11,5]
sol = Solution()
sol.canPartition(nums)

[[True, False, False, False, False, False, False, False, False, False, False, False], [True, True, False, False, False, False, False, False, False, False, False, False], [True, True, False, False, False, True, True, False, False, False, False, False], [True, True, False, False, False, True, True, False, False, False, False, True], [True, True, False, False, False, True, True, False, False, False, True, True]]


True

In [ ]:
# Optimised Dynamic Programming - Using 1D Array
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        # find sum of array elements
        total_sum = sum(nums)

        # if total_sum is odd, it cannot be partitioned into equal sum subsets
        if total_sum % 2 != 0:
            return False
        subset_sum = total_sum // 2

        # construct a dp table of size (subset_sum + 1)
        dp = [False] * (subset_sum + 1)
        dp[0] = True
        for curr in nums:
            for j in range(subset_sum, curr - 1, -1):
                dp[j] = dp[j] or dp[j - curr]

        return dp[subset_sum]

In [ ]:
nums = [1,5,11,5]
sol = Solution()
sol.canPartition(nums)

In [76]:
print([i for i in range(1,20,5)])

[1, 6, 11, 16]


# 997. Find the Town Judge
In a town, there are n people labeled from 1 to n. There is a rumor that one of these people is secretly the town judge.

If the town judge exists, then:

The town judge trusts nobody.
Everybody (except for the town judge) trusts the town judge.
There is exactly one person that satisfies properties 1 and 2.
You are given an array trust where trust[i] = [ai, bi] representing that the person labeled ai trusts the person labeled bi. If a trust relationship does not exist in trust array, then such a trust relationship does not exist.

Return the label of the town judge if the town judge exists and can be identified, or return -1 otherwise.

Example 1:

Input: n = 2, trust = [[1,2]]
Output: 2
Example 2:

Input: n = 3, trust = [[1,3],[2,3]]
Output: 3
Example 3:

Input: n = 3, trust = [[1,3],[2,3],[3,1]]
Output: -1
 

Constraints:

1 <= n <= 1000
0 <= trust.length <= 104
trust[i].length == 2
All the pairs of trust are unique.
ai != bi
1 <= ai, bi <= n

In [91]:
def findJudge(n, trust):

    adjList = {i: [] for i in range(1,n+1)}
    indegree = [0]*n
    outdegree = [0]*(n)
    for a,b in trust:
        indegree[b-1]+=1
        outdegree[a-1]+=1
    for i in range(n):
        if indegree[i] == n-1 and outdegree[i] == 0:
            return i+1
    return -1




In [92]:
n, trust = 2, [[1,2]]
findJudge(n, trust)

2

In [93]:
n , trust =3, [[1,3],[2,3],[3,1]]
findJudge(n, trust)

-1

In [94]:
n , trust = 3, [[1,3],[2,3]]
findJudge(n, trust)

3

In [ ]:
# Cleaner version: Complexity O(n+len(trust)) time and o(n) space
def findJudge(n, trust):
    indegree = [0] * (n + 1)
    outdegree = [0] * (n + 1)

    for a, b in trust:
        outdegree[a] += 1
        indegree[b] += 1

    for i in range(1, n+1):
        if indegree[i] == n - 1 and outdegree[i] == 0:
            return i
    return -1


## Note: Can There Be More Than One Town Judge?
the answer is... it's impossible!

If there were two town judges, then they would have to trust each other, otherwise we'd have a town judge not trusted by everybody. But this doesn't work, because town judges aren't supposed to trust anybody. Therefore, we know there can be at most one town judge.

## Note: Related Question

similar question on Leetcode, called Find the Celebrity. You need to do the same thing—find a person who has an indegree of N - 1 and an outdegree of 0. However, the input format is a bit different.

It's well worth a look at. A seemingly small difference (the input format) completely changes what the optimal algorithm to solve it is. Interestingly though, the optimal algorithm for that problem can also be used here. The only difference is that there, it has a cost of O(N), but here it has a cost of O(E). Try and figure out why once you've solved both problems. It's a really nice example of cost analysis with graphs.

Here’s how I’d answer that question in an interview or discussion setting:

---

Both **Find the Judge** and **Find the Celebrity** are essentially the same logical problem:

* Identify a single person who is **trusted/known by everyone else** (indegree = N - 1).
* And who **doesn’t trust/know anyone** (outdegree = 0).

### The Key Difference: Input Format

* **Find the Judge**: Input is given as an **edge list** (`trust` pairs).

  * You need to build indegree and outdegree by scanning through all trust pairs.
  * Complexity: **O(E)**, where *E* is the number of trust pairs.

* **Find the Celebrity**: Input is given as a **boolean API call** `knows(a, b)`.

  * You can’t precompute adjacency lists because you don’t have direct access to all edges.
  * The optimal strategy is a **candidate elimination algorithm**:

    1. Start with person 1 as the candidate.
    2. For each person `i`, if `candidate` knows `i`, then `candidate` cannot be celebrity → update candidate = `i`.
    3. After one pass, you get one potential candidate.
    4. Verify them by checking all `knows` relations.
  * Complexity: **O(N)** calls to `knows`, because you reduce candidates linearly.

### Why the Cost Difference?

* In **Find the Judge**, you’re given explicit edges, so the cost depends on how many trust relationships exist: **O(E)**. In the worst case, E = O(N²).
* In **Find the Celebrity**, you’re restricted to queries via `knows()`. The elimination trick means you only need **O(N)** queries instead of O(N²).

### The Takeaway

The “seemingly small difference” (direct edge list vs. query API) changes the optimal approach:

* **Judge** → adjacency/degree counting, cost depends on edges.
* **Celebrity** → candidate elimination, cost linear in people.

Interestingly, the **celebrity algorithm also works for the judge**:

* You could simulate the `knows()` API from the `trust` list, then apply the elimination process.
* But the cost would be **O(E)**, not O(N), because you’d have to scan the trust list each time you answer `knows(a, b)`.

---

👉 So the crux:

* **Same condition, different input model → different optimal complexity.**
* **Judge** = graph-based (O(E)).
* **Celebrity** = elimination-based (O(N)).

---

Perfect 👍 — here’s a side-by-side pseudo-code comparison you can use in an interview to clearly show the difference:

---

## 🏛 Find the Judge (edge list input)

```python
def findJudge(n, trust):
    indegree = [0] * (n + 1)
    outdegree = [0] * (n + 1)

    # Build degrees from trust pairs
    for a, b in trust:
        outdegree[a] += 1
        indegree[b] += 1

    # Judge = trusted by n-1 and trusts nobody
    for i in range(1, n + 1):
        if indegree[i] == n - 1 and outdegree[i] == 0:
            return i
    return -1
```

* **Complexity:** `O(E)` to scan all edges.
* **Space:** `O(N)` for arrays.

---

## 🎭 Find the Celebrity (API `knows(a, b)` input)

```python
def findCelebrity(n):
    # Step 1: Find candidate
    candidate = 0
    for i in range(1, n):
        if knows(candidate, i):
            candidate = i

    # Step 2: Verify candidate
    for i in range(n):
        if i == candidate: 
            continue
        if knows(candidate, i) or not knows(i, candidate):
            return -1
    return candidate
```

* **Complexity:** `O(N)` `knows()` calls.
* **Space:** `O(1)`.

---

### 🔑 Key Interview Line

* **Judge:** Explicit trust pairs → cost depends on edges (**O(E)**).
* **Celebrity:** Hidden graph via `knows()` API → candidate elimination trick makes it **O(N)**.
* Same logic (indegree = N - 1, outdegree = 0), but **input model drives the optimal algorithm**.

---

**one-liner answer**:

Both problems look for the person with indegree = N-1 and outdegree = 0. The key difference is input: in Find the Judge you’re given an edge list, so it’s O(E) with degree counting, while in Find the Celebrity you only have a knows(a,b) API, so the optimal elimination trick gives O(N). Same condition, but the input model changes the cost.


# 98. Validate Binary Search Tree

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

def isValidBST(root):
    def bst(node, low, high):
        if not node:
            return True
        if (low is not None and node.val<=low ) or (high is not None and node.val>=high):
            return False
        return bst(node.left, low, node.val) and bst(node.right, node.val, high)
    return bst(root, None, None)
            


# 2371. Minimize Maximum Value in a Grid

You are given an m x n integer matrix grid containing distinct positive integers.

You have to replace each integer in the matrix with a positive integer satisfying the following conditions:

The relative order of every two elements that are in the same row or column should stay the same after the replacements.
The maximum number in the matrix after the replacements should be as small as possible.
The relative order stays the same if for all pairs of elements in the original matrix such that grid[r1][c1] > grid[r2][c2] where either r1 == r2 or c1 == c2, then it must be true that grid[r1][c1] > grid[r2][c2] after the replacements.

For example, if grid = [[2, 4, 5], [7, 3, 9]] then a good replacement could be either grid = [[1, 2, 3], [2, 1, 4]] or grid = [[1, 2, 3], [3, 1, 4]].

Return the resulting matrix. If there are multiple answers, return any of them.

 

Example 1:


Input: grid = [[3,1],[2,5]]
Output: [[2,1],[1,2]]
Explanation: The above diagram shows a valid replacement.
The maximum number in the matrix is 2. It can be shown that no smaller value can be obtained.
Example 2:

Input: grid = [[10]]
Output: [[1]]
Explanation: We replace the only number in the matrix with 1.
 

Constraints:

m == grid.length
n == grid[i].length
1 <= m, n <= 1000
1 <= m * n <= 105
1 <= grid[i][j] <= 109
grid consists of distinct integers.

Hint 1
- Can you think of which element in the grid you should replace first?

Hint 2
- Replace the elements in the matrix from the smallest number to the largest. Replace each element with the smallest possible number so far.


In [ ]:
# Ali's solution using BFS (looks satisfying the asked outcome but not correct, as per ChatGPT)
import math
from collections import deque
def minScore(grid):
    directions = [(0,1),(0,-1),(1,0),(-1,0)]
    m = len(grid)
    n = len(grid[0])
    minVal = (math.inf,0,0)

    for i in range(m):
        for j in range(n):
            if grid[i][j] < minVal[0]:
                minVal = (grid[i][j],i,j)
    smallest = 1
    first, i, j = minVal
    queue = deque([(1,i,j)])
    seen = [first]
    grid_res = [[None]*n for _ in range(m)]
    grid_res[i][j] = smallest
    while queue:
        num,i,j = queue.popleft()
        print(num, i, j, seen, grid_res)
        
        for dir_i,dir_j in directions:
            ii = i+dir_i
            jj = j+dir_j
            if 0<=ii<=m-1 and 0<=jj<=n-1 and grid[ii][jj] not in seen:
                
                if grid[ii][jj]>num:
                    grid_res[ii][jj] = num+1
                    
                else:
                    grid_res[ii][jj] = num-1


                seen.append(grid[ii][jj])
                queue.append((grid_res[ii][jj],ii,jj))
            
        
    return grid_res


In [141]:
grid = [[3,1],[2,5]]
minScore(grid)

1 0 1 [1] [[None, 1], [None, None]]
2 0 0 [1, 3, 5] [[2, 1], [None, 2]]
2 1 1 [1, 3, 5, 2] [[2, 1], [1, 2]]
1 1 0 [1, 3, 5, 2] [[2, 1], [1, 2]]


[[2, 1], [1, 2]]

In [142]:
grid = [[2, 4, 5], [7, 3, 9]]
minScore(grid)

1 0 0 [2] [[1, None, None], [None, None, None]]
2 0 1 [2, 4, 7] [[1, 2, None], [2, None, None]]
2 1 0 [2, 4, 7, 5, 3] [[1, 2, 3], [2, 3, None]]
3 0 2 [2, 4, 7, 5, 3] [[1, 2, 3], [2, 3, None]]
3 1 1 [2, 4, 7, 5, 3, 9] [[1, 2, 3], [2, 3, 4]]
4 1 2 [2, 4, 7, 5, 3, 9] [[1, 2, 3], [2, 3, 4]]


[[1, 2, 3], [2, 3, 4]]

In [153]:
# Ali's solution, using sorted-rank assignment
def minScore(grid):
    m = len(grid)
    n = len(grid[0])
    sorted_list = []
    for i in range(m):
        for j in range(n):
            val = grid[i][j]
            sorted_list.append((val,i,j))
    sorted_list.sort(key=lambda x:x[0])
    row_max = [0]*m
    col_max = [0]*n
    result = [[None]*n for _ in range(m)]
    for val, i, j in sorted_list:
        new_val = 1 + max(row_max[i], col_max[j])
        row_max[i] = max(row_max[i], new_val)
        col_max[j] = max(col_max[j], new_val)
        result[i][j] = new_val

    return result


In [154]:
grid = [[2, 4, 5], [7, 3, 9]]
minScore(grid)

[[1, 2, 3], [2, 1, 4]]

In [ ]:
# Leetcode solution 1: time complexity: O(m.n.log(m.n)), space complexity: O(m.n). 
# Also: In Python, the sort method sorts a list using the Timsort algorithm which 
# is a combination of Merge Sort and Insertion Sort and has O(l) additional space, 
# where l is the size of the list.

class Solution:
    def minScore(self, grid: List[List[int]]) -> List[List[int]]:
        row, col = len(grid), len(grid[0])

        nums = []
        # Create rows and cols to store the minimum values for every row and column.
        rows = [1 for i in range(row)]
        cols = [1 for i in range(col)]

        # Create a matrix nums that stores the values of the matrix and their indices.
        for i in range(row):
            for j in range(col):
                nums.append((grid[i][j], i, j))

        nums.sort()
        for tup in nums:
            _, i, j = tup
            # Find the maximum value of rows[i] and cols[j] till now and assign it to val.
            val = max(rows[i], cols[j])
            grid[i][j] = val
            # Update the new maximum value in rows[i] and cols[j].
            rows[i], cols[j] = val + 1, val + 1
        return grid

In [ ]:
# Approach 2: Min-heap instead of sorting. Complexity is the same, tho.
import heapq
class Solution:
    def minScore(self, grid: List[List[int]]) -> List[List[int]]:
        row_size, col_size = len(grid), len(grid[0])

        # Min-heap to store elements with their values and coordinates
        min_heap = []
        # Initialize rows and cols arrays to keep track of maximum values assigned
        rows = [1] * row_size
        cols = [1] * col_size

        # Populate the min-heap with all elements of the grid
        for i in range(row_size):
            for j in range(col_size):
                heapq.heappush(min_heap, (grid[i][j], i, j))

        # Process elements in ascending order of their values
        while min_heap:
            _, i, j = heapq.heappop(min_heap)
            # Determine the smallest assignable value based on rows and cols constraints
            val = max(rows[i], cols[j])
            grid[i][j] = val
            # Update rows and cols arrays with the next possible value for each row and column
            rows[i] = val + 1
            cols[j] = val + 1

        return grid